In [1]:
import cv2
import numpy as np

In [2]:
cap = cv2.VideoCapture(0)
ret, frame = cap.read()

#Loads the classifier
face_cascade = cv2.CascadeClassifier('Computer-Vision-with-Python\DATA\haarcascades\haarcascade_frontalface_default.xml')

# Detect objects of different sizes in the input image. The detected objects are returned as a list of rectangles.
rects = face_cascade.detectMultiScale(frame)


(face_x,face_y,w,h) = tuple(rects[0])
track_window = (face_x,face_y,w,h)

roi = frame[face_y:face_y+h,face_x:face_x+w]  #roi - Region Of Interest


# RGB components of an image are all correlated with the amount of light hitting the object.
# Image descriptions in terms of those components make object discrimination difficult. 
# Describing in terms of hue/lightness/chroma or hue/lightness/saturation prove to be more relevant.
hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

#Calculate Histogram 
#[hsv_roi] is thre source array whose Histogram is to be calculated
#[0] is the channel
#Mask is None
#Number of bins = 180
#Range is [0,180]

roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)


#cv2.TERM_CRITERIA_EPS - stop the algorithm iteration if specified accuracy, epsilon, is reached. (Here 10)
#cv2.TERM_CRITERIA_MAX_ITER - stop the algorithm after the specified number of iterations, max_iter.(Here 1)
#cv2.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER - stop the iteration when any of the above condition is met.

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)   


while True :
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)          
        # [hsv] is the source
        # roi_hist is the model
        
        ret, track_window = cv2.meanShift(dst,track_window,term_crit)
        
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),5)
        cv2.imshow('img',img2)
        
        k = cv2.waitKey(1) & 0xFF
        if k == 27:                            #Press Esc to exit
            break
    else:
            break
            
cv2.destroyAllWindows()
cap.release()